In [1]:
import $exec.spark_functionality


Compiling /home/jovyan/work/etl_textreuse/spark_functionality.scCompiling /home/jovyan/work/etl_textreuse/spark_functionality.sc #2

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark UI

import $exec.$                  


# 1. create piecewise reuses

Create version of the reuse data where there is a pieces table recording each piece in a reuse in each text, linked by a textreuses table.

In [2]:
val textreuses1 = get_a3s("textreuses")

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

textreuses1: DataFrame = [textreuse_id: bigint, trs1_id: bigint ... 7 more fields]

In [2]:
// val rdd = sc.parallelize(textreuses1.take(1000))
// val rddDf = spark.createDataFrame(rdd,schema=textreuses1.schema)
// val textreuses = materialise_local("textreuses",df=rddDf)

In [3]:
val pieces = materialise_a3s(
    "orig_pieces", 
    spark.sql("""
    SELECT 
        textreuse_id*2-1 AS piece_id, 
        trs1_id AS trs_id,
        trs1_start AS trs_start,
        trs1_end AS trs_end 
    FROM textreuses
    UNION ALL
    SELECT 
        textreuse_id*2 AS piece_id,
        trs2_id AS trs_id,
        trs2_start AS trs_start,
        trs2_end AS trs_end 
    FROM textreuses
"""))
// val textreuses = materialise_a3s("textreuses", spark.sql("""
// SELECT tr_id, tr_id*2-1 AS piece1_id, tr_id*2 AS piece2_id, align_length, positives_percent FROM textreuses_a
// """))

parquet at spark_functionality.sc:120 
 (kill)

0 / 424

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

pieces: DataFrame = [piece_id: bigint, trs_id: bigint ... 2 more fields]

# 2. map overlapping pieces to one id

Create an initial clustering of the piece_ids by mapping all pieces that overlap sufficiently in a given text to one id

In [4]:
import spark.implicits._
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}
import org.apache.spark.sql.catalyst.encoders.RowEncoder

val input_type = StructType(Seq(
    StructField("trs_start",IntegerType),
    StructField("trs_end",IntegerType),
    StructField("piece_id",IntegerType)
))

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.expressions.Aggregator

object GetPieceIdMapping extends Aggregator[Row, ArrayBuffer[(Int,Int,Int)], Int] {
  import org.apache.spark.sql.catalyst.encoders.ExpressionEncoder

  // A zero value for this aggregation. Should satisfy the property that any b + zero = b
  def zero: ArrayBuffer[(Int,Int,Int)] = ArrayBuffer()
  // Combine two values to produce a new value. For performance, the function may modify `buffer`
  // and return it instead of constructing a new object
  def reduce(to_check: ArrayBuffer[(Int,Int,Int)], row: Row): ArrayBuffer[(Int,Int,Int)] = {
      val t_start = row.getInt(0)
      val idx = to_check.indexWhere(_._1 >= t_start - 180)
      if (idx < 0) to_check.clear() else to_check.remove(0, idx)
      to_check += ((t_start,row.getInt(1),row.getInt(2)))
      to_check
  }
  // Merge two intermediate values
  def merge(b1: ArrayBuffer[(Int,Int,Int)], b2: ArrayBuffer[(Int,Int,Int)]): ArrayBuffer[(Int,Int,Int)] = {
//      b1 ++= b2 & sort
      throw new UnsupportedOperationException()
  }
  // Transform the output of the reduction
  def finish(to_check: ArrayBuffer[(Int,Int,Int)]): Int = {
      val (t_start, t_end, piece_id) = to_check.last
      to_check.find(r => {
          val limit = math.min(
              math.max(
                  math.min(t_end - t_start, r._2 - r._1)
                  /4, 10),
              180)
          math.abs(r._1 - t_start) <= limit && math.abs(r._2 - t_end) <= limit
      }).get._3
  }
  // Specifies the Encoder for the intermediate value type
  def bufferEncoder: Encoder[ArrayBuffer[(Int,Int,Int)]] = ExpressionEncoder()
  // Specifies the Encoder for the final output value type
  def outputEncoder: Encoder[Int] = Encoders.scalaInt
}

// Register the function to access it
spark.udf.register("get_piece_id_mapping", functions.udaf(GetPieceIdMapping,RowEncoder(input_type)))

import spark.implicits._

import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}

import org.apache.spark.sql.catalyst.encoders.RowEncoder


input_type: StructType = StructType(
  StructField("trs_start", IntegerType, true, {}),
  StructField("trs_end", IntegerType, true, {}),
  StructField("piece_id", IntegerType, true, {})
)
import scala.collection.mutable.ArrayBuffer

import org.apache.spark.sql.expressions.Aggregator


defined object GetPieceIdMapping
res3_7: expressions.UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cmd3$Helper$GetPieceIdMapping$@68465094,
  ExpressionEncoder(
    If(
      IsNull(
        BoundReference(0, ObjectType(interface org.apache.spark.sql.Row), true)
      ),
      Literal(
        null,
        StructType(
          StructField("trs_start", IntegerType, true, {}),
          StructField("trs_end", IntegerType, true, {}),
          StructField("piece_id", IntegerType, true, {})
        )
      ),
      CreateNam

In [5]:
val piece_id_mappings = materialise_a3s("piece_id_mappings_tmp", spark.sql("""
SELECT piece_id AS orig_piece_id, get_piece_id_mapping(trs_start,trs_end,piece_id) OVER (PARTITION BY trs_id ORDER BY trs_start, piece_id) AS defrag_mapping
FROM orig_pieces
"""))

parquet at spark_functionality.sc:120 
 (kill)

0 / 117

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_mapping: int]

We convert the mapping ids into monotonically increasing ids 


In [6]:
val degrag_id_mapping = materialise_row_numbers(
  "defrag_id_mapping",
  spark.sql("""
    SELECT DISTINCT defrag_mapping
    FROM piece_id_mappings_tmp
    """),
  "defrag_piece_id"
)

parquet at spark_functionality.sc:120 
 (kill)

0 / 78

0 / 40

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

0 / 39

parquet at spark_functionality.sc:120 
 (kill)

0 / 40

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

Add the piece id mapping back to piece_id

In [7]:
val updated_piece_id_mappings = materialise_a3s("piece_id_mappings", spark.sql("""
SELECT orig_piece_id, defrag_piece_id FROM piece_id_mappings_tmp
INNER JOIN defrag_id_mapping USING(defrag_mapping)
"""))

parquet at spark_functionality.sc:120 
 (kill)

0 / 78

parquet at spark_functionality.sc:120 
 (kill)

0 / 40

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

updated_piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_piece_id: bigint]

Delete the temporary mapping data

In [8]:
delete_s3("piece_id_mappings_tmp")
delete_s3("defrag_id_mapping")

Here, we create a filtered version of the pieces table based on the overlap clustering:

In [9]:
val defrag_pieces = materialise_a3s("defrag_pieces", spark.sql("""
SELECT defrag_piece_id AS piece_id, trs_id, MIN(trs_start) AS trs_start, MAX(trs_end) AS trs_end
FROM piece_id_mappings
INNER JOIN orig_pieces ON piece_id = orig_piece_id
GROUP BY defrag_piece_id, trs_id
"""))

parquet at spark_functionality.sc:120 
 (kill)

0 / 67

parquet at spark_functionality.sc:120 
 (kill)

0 / 117

parquet at spark_functionality.sc:120 
 (kill)

...

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

defrag_pieces: DataFrame = [piece_id: bigint, trs_id: bigint ... 2 more fields]

Create textreuses in terms of mapping between pieces

In [10]:
val orig_textreuses = materialise_a3s("orig_textreuses", spark.sql("""
SELECT textreuse_id, textreuse_id*2-1 AS piece1_id, textreuse_id*2 AS piece2_id, align_length, positives_percent FROM textreuses
"""))

0 / 212

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

orig_textreuses: DataFrame = [textreuse_id: bigint, piece1_id: bigint ... 3 more fields]

Create the textreuse edg

In [11]:
val defrag_textreuses = materialise_row_numbers(
    "defrag_textreuses",
    spark.sql("""
    SELECT
        pm1.defrag_piece_id AS piece1_id, 
        pm2.defrag_piece_id AS piece2_id,
        COUNT(*) AS num_orig_links
    FROM orig_textreuses tr
    LEFT JOIN piece_id_mappings pm1 ON (tr.piece1_id=pm1.orig_piece_id)
    LEFT JOIN piece_id_mappings pm2 ON (tr.piece2_id=pm2.orig_piece_id)
    GROUP BY pm1.defrag_piece_id,pm2.defrag_piece_id
    """),
    "textreuse_id"
)

parquet at spark_functionality.sc:120 
 (kill)

0 / 55

0 / 67

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

...

0 / 99

parquet at spark_functionality.sc:120 
 (kill)

0 / 100

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

In [12]:
defrag_pieces.count()

count at cmd11.sc:1 
 (kill)

0 / 50

0 / 1

res11: Long = 374683929L

In [14]:
val defrag_textreuses = get_a3s("defrag_textreuses")
defrag_textreuses.count()

0 / 1

count at cmd13.sc:2 
 (kill)

0 / 100

0 / 1

defrag_textreuses: DataFrame = [textreuse_id: bigint, piece1_id: bigint ... 2 more fields]
res13_1: Long = 965043090L

# 4. create the array of link indices for clustering

Create a parquet file containing the reuse network in the form where each array index contains the indices of other nodes it links to

In [ ]:
// val textreuse_graph = materialise_a3s("textreuse_graph",spark.sql("""
// mapped_textreuses AS (
//   SELECT pm1.new_new_piece_id AS piece1_mid, pm2.new_new_piece_id AS piece2_mid
//   FROM textreuses tr
//   LEFT JOIN piece_id_mappings pm1 ON (tr.piece1_id=pm1.orig_piece_id)
//   LEFT JOIN orig_piece_ids_to_new_new_piece_ids pm2 ON (tr.piece2_id=pm2.orig_piece_id)
// ),
// mapped_graph AS (
//     SELECT DISTINCT piece1_mid, piece2_mid FROM (
//       SELECT piece1_mid, piece2_mid FROM mapped_textreuses
//       UNION ALL
//       SELECT piece2_mid AS piece1_mid, piece1_mid AS piece2_mid FROM mapped_textreuses
//     )
// )
// SELECT /*+ COALESCE(1) */ collect_list(piece2_mid) AS edges
// FROM mapped_graph
// GROUP BY piece1_mid
// ORDER BY piece1_mid
// """))

# 5. clustering 

Done separately with either the Python or Go versionn of the Chinese Whispers algorithm